In [111]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [114]:
from manim import *

In [ ]:
%%manim -qm -v WARNING ThreeD

config.media_width = "60%"

class ThreeD(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes(
            x_range=(-5, 5, 1), 
            y_range=(-5, 5, 1), 
            z_range=(-5, 5, 1), 
            x_length=10.5, 
            y_length=10.5, 
            z_length=10.5,
            axis_config={"include_numbers": True}
        )
        flat = Surface(
            lambda u, v: axes.coords_to_point(
                u,
                v,
                0
            ), v_range=[0, TAU], u_range=[-PI/2, PI/2],
            checkerboard_colors=[BLUE, WHITE], resolution=(15, 32)
        )
        sphere = Surface(
            lambda u, v: axes.coords_to_point(
                1.5 * np.cos(u) * np.cos(v),
                1.5 * np.cos(u) * np.sin(v),
                1.5 * np.sin(u)
            ), v_range=[0, TAU], u_range=[-PI/2, PI/2],
            checkerboard_colors=[BLUE, WHITE], resolution=(15, 32)
        )
        trans = Surface(
            lambda u, v: axes.coords_to_point(
                2 * (1.5 * np.cos(u) * np.cos(v)) * (1.5 * np.sin(u)),
                3 * (1.5 * np.cos(u) * np.sin(v)) * (1.5 * np.sin(u)),
                5 * (1.5 * np.sin(u)) - 5/2 * (1.5 * np.sin(u)) ** 2
            ), v_range=[0, TAU], u_range=[-PI/2, PI/2],
            checkerboard_colors=[BLUE, WHITE], resolution=(15, 32)
        )
        #self.renderer.camera.light_source.move_to(3*IN) # changes the source of the light
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        self.play(FadeIn(axes))
        self.play(Create(flat))
        self.wait(2)
        self.play(ReplacementTransform(flat, sphere), run_time=4)
        self.wait(3)
        self.play(ReplacementTransform(sphere.copy(), trans.copy()), run_time=4)
        self.wait(8)

In [ ]:
%%manim -qm -v WARNING OpeningManim

config.media_width = "60%"

class OpeningManim(Scene):
    def construct(self):
        grid = NumberPlane()
        grid_title = Tex("Linear Transformation", font_size=52)
        grid_title.to_corner(UP + LEFT)

        self.add(grid, grid_title)  # Make sure title is on top of grid
        self.play(
            FadeIn(grid_title, shift=UP),
            Create(grid, run_time=3, lag_ratio=0.1),
        )
        self.wait()
        arrow_1a = Arrow(start=np.array([0,0,0]), end=np.array([0,1,0]),buff=0, color=GOLD)
        arrow_2a = Arrow(start=np.array([0,0,0]), end=np.array([1,0,0]),buff=0, color=GOLD)
        arrow_1b = Arrow(start=np.array([0,0,0]), end=np.array([2,-1,0]),buff=0, color=GOLD)
        arrow_2b = Arrow(start=np.array([0,0,0]), end=np.array([2,2,0]),buff=0, color=GOLD)
        Matrix1 = Matrix([[2, 2], [2, -1]])
        self.play(Create(arrow_1a),Create(arrow_2a))
        vek_1a = MathTex(r"\vec{i}").next_to(arrow_1a.get_end(), RIGHT, buff=0.3)
        vek_2a = MathTex(r"\vec{j}").next_to(arrow_2a.get_end(), RIGHT, buff=0.3)
        vek_1b = MathTex(r"\vec{u}").next_to(arrow_1a.get_end(), RIGHT, buff=0.3)
        vek_2b = MathTex(r"\vec{v}").next_to(arrow_2a.get_end(), RIGHT, buff=0.3)
        vek_1b.add_updater(lambda x : x.move_to(vek_1a))
        vek_2b.add_updater(lambda x : x.move_to(vek_2a))
        Vector1 = MobjectMatrix([[MathTex(r'\vec{u}')],[MathTex(r'\vec{v}')]]).to_corner(UL)
        equal = MathTex(r"=").next_to(Vector1, RIGHT)
        self.play(Write(vek_1a), Write(vek_2a))
        Matrix1.next_to(equal, RIGHT)
        Vector2 = MobjectMatrix([[MathTex(r'\vec{i}')], [MathTex(r'\vec{j}')]]).next_to(Matrix1, RIGHT)
        self.play(Transform(grid_title, Matrix1))
        self.play(Create(Vector1), Create(Vector2), Create(equal))
        self.wait()
        grid.prepare_for_nonlinear_transform()
        self.play(
            grid.animate.apply_function(
                lambda p: np.array([2*p[0] + 2*p[1], 2*p[0] - p[1], 0])
            ),
            ReplacementTransform(arrow_1a,arrow_1b),
            ReplacementTransform(arrow_2a,arrow_2b),
            vek_1a.animate.next_to(arrow_1b.get_end(), RIGHT, buff=0.3),
            vek_2a.animate.next_to(arrow_2b.get_end(), RIGHT, buff=0.3),
            ReplacementTransform(vek_1a, vek_1b),
            ReplacementTransform(vek_2a, vek_2b),
            run_time=3,
        )
        self.wait()
        VEK1 = Matrix1.get_columns()[1]
        VEK2 = Matrix1.get_columns()[0]
        VEKTa = Matrix([[2],[-1]]).next_to(arrow_1b.get_end(), RIGHT, buff=0.3)
        VEKTb = Matrix([[2],[2]]).next_to(arrow_2b.get_end(), RIGHT, buff=0.3)
        self.play(ReplacementTransform(vek_1b,VEKTa))
        self.play(ReplacementTransform(VEK1.copy(),VEKTa.copy()))
        self.wait()
        self.play(ReplacementTransform(vek_2b,VEKTb))
        self.play(ReplacementTransform(VEK2.copy(),VEKTb.copy()))
        self.wait()

In [ ]:
%%manim -qm -v WARNING ThreeD2

config.media_width = "60%"

class ThreeD2(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes(
            x_range=(-5, 5, 1), 
            y_range=(-5, 5, 1), 
            z_range=(-5, 5, 1), 
            x_length=10.5, 
            y_length=10.5, 
            z_length=10.5,
            axis_config={"include_numbers": True}
        )
        flat = Surface(
            lambda u, v: axes.coords_to_point(
                u,
                v,
                0
            ), v_range=[-PI, PI], u_range=[-PI, PI],
            checkerboard_colors=[BLUE, WHITE], resolution=(20, 20)
        )
        sphere = Surface(
            lambda u, v: axes.coords_to_point(
                (3 + np.cos(u+PI)) * np.cos(v+PI),
                (3 + np.cos(u+PI)) * np.sin(v+PI),
                np.sin(u+PI)
            ),v_range=[-PI, PI], u_range=[-PI, PI],
            checkerboard_colors=[BLUE, WHITE], resolution=(20, 20)
        )
        #self.renderer.camera.light_source.move_to(3*IN) # changes the source of the light
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        self.add(axes,flat)
        self.wait(3)
        self.play(ReplacementTransform(flat, sphere), run_time=4)
        self.wait(4)

In [ ]:
%%manim -qm -v WARNING Project

config.media_width = "80%"

class Project(Scene):
    def construct(self):
        def SEIQR(t, yvek, N, beta, mu, epsilon, nu, gamma, xi, phi):
            S, E, I, Q, R = yvek[0], yvek[1], yvek[2], yvek[3], yvek[4]
            dS = -beta*S*I + mu*N - mu*S
            dE = beta*S*I - nu*E - epsilon*E - mu*E
            dI = nu*E - mu*I - xi*I - gamma*I
            dQ = xi*I - phi*Q - mu*Q
            dR = epsilon*E + gamma*I + phi*Q - mu*R
            return dS, dE, dI, dQ, dR

        def RK4(initial, lower, upper, h, N, beta, mu, epsilon, nu, gamma, xi, phi):
            t = np.arange(lower, upper, h)
            n, dim = len(t), len(initial)
            yvek = np.zeros([n, dim])
            k1v, k2v, k3v, k4v = np.zeros([n, dim]), np.zeros([n, dim]), np.zeros([n, dim]), np.zeros([n, dim])
            yvek[0, :] = initial
            for i in range(n-1):
              k1v[i, :] = h * SEIQR(t[i], yvek[i, :], N, beta, mu, epsilon, nu, gamma, xi, phi)
              k2v[i, :] = h * SEIQR(t[i] + (1/2)*h, yvek[i, :] + (1/2)*k1v[i, :], N, beta, mu, epsilon, nu, gamma, xi, phi)
              k3v[i, :] = h * SEIQR(t[i] + (1/2)*h, yvek[i, :] + (1/2)*k2v[i, :], N, beta, mu, epsilon, nu, gamma, xi, phi)
              k4v[i, :] = h * SEIQR(t[i] + h, yvek[i, :] + k3v[i, :], N, beta, mu, epsilon, nu, gamma, xi, phi)
              yvek[i+1, :] = yvek[i, :] + (1/6)*(k1v[i, :] + 2*k2v[i, :] + 2*k3v[i, :] + k4v[i, :])
            return yvek
        
        ax = NumberPlane(
            x_range=[-20, 200, 20],
            y_range=[-10000, 110000, 10000],
            x_length=11,
            y_length=8,
            background_line_style={
                "stroke_color": BLUE,
                "stroke_width": 3,
                "stroke_opacity": 0.2
            },
            axis_config={
                "include_numbers": True
            },
            y_axis_config={
                "label_direction": DL
            }
        ).to_edge(LEFT)
        self.add_sound("otherside.mp3")
        N = 100000
        beta, mu, epsilon, nu, gamma, xi, phi = 0.0000005, 0.0000003, 0.0000003, 0.05, 0.01, 0.05, 0.05
        t_min, t_max = 0, 200
        h = 1
        L = len(np.arange(t_min, t_max, h))
        COLOR = [BLUE, YELLOW, RED, PURPLE, GREEN]
        y0_A = [40000, 30000, 30000, 0, 0]
        y_vals_A = RK4(y0_A, t_min, t_max, h, N, beta, mu, epsilon, nu, gamma, xi, phi)
        func_A = []
        for i in range(len(y0_A)):
          func = ParametricFunction(lambda t: ax.coords_to_point(
              t*h,
              y_vals_A[np.uint(t), i],
              0
              ), color=COLOR[i], t_range = np.array([0, L-1, 1])
          )
          func_A.append(func)
        self.add(ax)
        line1 = MathTex(r"&N = 100000 \\ \\",
                        r"&S(0) = 40000 \\",
                        r"&E(0) = 30000 \\",
                        r"&I(0) = 30000 \\",
                        r"&Q(0) = 0 \\",
                        r"&R(0) = 0 \\ \\",
                        r"&\beta = 0.0000005 \\",
                        r"&\varepsilon = 0.0000003 \\",
                        r"&\mu = 0.0000003 \\",
                        r"&\nu = 0.05 \\",
                        r"&\gamma = 0.01 \\",
                        r"&\xi = 0.05 \\",
                        r"&\phi = 0.05", font_size = 25).to_corner(UR)
        for i,sub in enumerate(line1):
          if i<=5 and i>=1:
            sub.set_color(COLOR[i-1])
        self.play(Write(line1), run_time=3)
        self.wait()
        self.play(Create(func_A[0]),
                  Create(func_A[1]),
                  Create(func_A[2]),
                  Create(func_A[3]),
                  Create(func_A[4]),
                  run_time=3)
        self.wait(2)
        y0_B = [40000, 30000, 30000, 0, 0]
        beta = 0.000004
        y_vals_B = RK4(y0_B, t_min, t_max, h, N, beta, mu, epsilon, nu, gamma, xi, phi)
        func_B = []
        for i in range(len(y0_B)):
          func = ParametricFunction(lambda t: ax.coords_to_point(
              t*h,
              y_vals_B[np.uint(t), i],
              0
              ), color=COLOR[i], t_range = np.array([0, L-1, 1])
          )
          func_B.append(func)
        line2 = MathTex(r"&N = 100000 \\ \\",
                        r"&S(0) = 40000 \\",
                        r"&E(0) = 30000 \\",
                        r"&I(0) = 30000 \\",
                        r"&Q(0) = 0 \\",
                        r"&R(0) = 0 \\ \\",
                        r"&\beta = 0.000004 \\",
                        r"&\varepsilon = 0.0000003 \\",
                        r"&\mu = 0.0000003 \\",
                        r"&\nu = 0.05 \\",
                        r"&\gamma = 0.01 \\",
                        r"&\xi = 0.05 \\",
                        r"&\phi = 0.05", font_size = 25).to_corner(UR)
        for i,sub in enumerate(line2):
          if i<=5 and i>=1:
            sub.set_color(COLOR[i-1])
        self.play(Transform(func_A[0], func_B[0]),
                  Transform(func_A[1], func_B[1]),
                  Transform(func_A[2], func_B[2]),
                  Transform(func_A[3], func_B[3]),
                  Transform(func_A[4], func_B[4]),
                  Transform(line1[6], line2[6]),
                  run_time=3)
        self.wait(2)
        y0_C = [99000, 500, 500, 0, 0]
        y_vals_C = RK4(y0_C, t_min, t_max, h, N, beta, mu, epsilon, nu, gamma, xi, phi)
        func_C = []
        for i in range(len(y0_C)):
          func = ParametricFunction(lambda t: ax.coords_to_point(
              t*h,
              y_vals_C[np.uint(t), i],
              0
              ), color=COLOR[i], t_range = np.array([0, L-1, 1])
          )
          func_C.append(func)
        line3 = MathTex(r"&N = 100000 \\ \\",
                        r"&S(0) = 99000 \\",
                        r"&E(0) = 500 \\",
                        r"&I(0) = 500 \\",
                        r"&Q(0) = 0 \\",
                        r"&R(0) = 0 \\ \\",
                        r"&\beta = 0.000004 \\",
                        r"&\varepsilon = 0.0000003 \\",
                        r"&\mu = 0.0000003 \\",
                        r"&\nu = 0.05 \\",
                        r"&\gamma = 0.01 \\",
                        r"&\xi = 0.05 \\",
                        r"&\phi = 0.05", font_size = 25).to_corner(UR)
        for i,sub in enumerate(line3):
          if i<=5 and i>=1:
            sub.set_color(COLOR[i-1])
        self.play(Transform(func_A[0], func_C[0]),
                  Transform(func_A[1], func_C[1]),
                  Transform(func_A[2], func_C[2]),
                  Transform(func_A[3], func_C[3]),
                  Transform(func_A[4], func_C[4]),
                  Transform(line1[1], line3[1]),
                  Transform(line1[2], line3[2]),
                  Transform(line1[3], line3[3]),
                  run_time=3)
        self.wait(2)
        self.play(FadeOut(func_A[0]),
                  FadeOut(func_A[1]),
                  FadeOut(func_A[3]),
                  FadeOut(func_A[4]),
                  )
        Fun = func_A[2]
        tracker = ValueTracker(0.05)
        self.remove(func_A[2])
        Fun.add_updater(lambda x : x.become(ParametricFunction(lambda t: ax.coords_to_point(
              t*h,
              RK4(y0_C, t_min, t_max, h, N, beta, mu, epsilon, nu, gamma, tracker.get_value(), phi)[np.uint(t), 2],
              0
              ), color=RED, t_range = np.array([0, L-1, 1])
          )))
        specific = line1[11]
        self.remove(line1[11])
        specific.add_updater(lambda x : x.become(MathTex(
                        r"&\xi = " + str(round(tracker.get_value(),4)) + r" \\", font_size = 25).move_to(line1[11])))
        self.add(line1[11], Fun)
        self.wait()
        self.play(tracker.animate.set_value(0), run_time=1.5)
        self.wait()
        self.play(tracker.animate.set_value(0.1), run_time=1.5)
        self.wait()
        self.play(tracker.animate.set_value(0.05), run_time=1.5)
        self.wait()




Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%%manim -qm -v WARNING Project2

config.media_width = "80%"
import random

class Project2(Scene):
    def construct(self):
        ax = NumberPlane(
            x_range=[0, 10, 1],
            y_range=[0, 1.025, 0.1],
            x_length=9.5,
            y_length=5.8,
            background_line_style={
                "stroke_width": 0,
                "stroke_opacity": 0
            },
            y_axis_config={"label_direction": DL,
                           "numbers_to_include": [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}
        ).to_edge(LEFT)
        self.add_sound("aria_math.mp3")
        title = Tex("Probability of Head").to_edge(UP)
        x_label = MathTex(r"P(H=x)", font_size=32).next_to(ax, DOWN)
        self.add(ax)
        self.play(Write(title))
        self.play(Write(x_label))
        Head, Tail = 0, 0
        pic, tex, grup = VGroup(), VGroup(), VGroup()
        Number, Height = np.zeros([10]), np.zeros([10])
        Total = 0
        for i in range(10):
            x = random.randint(0,1)
            if x==0:
              Head = Head + 1
              pic.add(Circle(radius=0.2, color=BLUE))
              tex.add(Tex("H", font_size=22))
            else :
              Tail = Tail + 1
              pic.add(Circle(radius=0.2, color=RED))
              tex.add(Tex("T", font_size=22))
            grup.add(VGroup(pic[i], tex[i]))
        Total += 1
        grup.arrange(DOWN)
        grup.to_edge(RIGHT, buff=1)
        for JAM in grup:
          self.play(Create(JAM))
        self.wait()
        count = MathTex(r"P(H)=\dfrac{"+str(Head)+r"}{"+str(Head+Tail)+r"}", font_size=34).next_to(grup, LEFT)
        self.play(Write(count))
        self.wait()
        rect = VGroup()
        for i in range(10):
          if Head == i+1:
            Number[i] += 1
          Height[i] = Number[i]/Total
          poly = [ax.coords_to_point(i,0,0),
                  ax.coords_to_point(i+1,0,0),
                  ax.coords_to_point(i+1,Height[i],0),
                  ax.coords_to_point(i,Height[i],0)]
          temp = Polygon(*poly).set_fill(BLUE, opacity=0.5)
          num = Tex(str(i+1), font_size=28).next_to(temp, UP, buff=0.1)
          V = VGroup(temp, num)
          rect.add(V)
        self.play(Create(rect), run_time=4)
        self.wait()
        NUM = Tex("N="+str(1), font_size=34).next_to(count, DOWN)
        self.play(Write(NUM))
        self.wait()
        for N in range(199):
          self.remove(rect, count, grup, NUM)
          Head, Tail = 0, 0
          for i in range(10):
              x = random.randint(0,1)
              if x==0:
                Head = Head + 1
                pic[i] = Circle(radius=0.2, color=BLUE)
                tex[i] = Tex("H", font_size=22)
              else :
                Tail = Tail + 1
                pic[i] = Circle(radius=0.2, color=RED)
                tex[i] = Tex("T", font_size=22)
              grup[i] = VGroup(pic[i], tex[i])
          Total += 1
          grup.arrange(DOWN)
          grup.to_edge(RIGHT, buff=1)
          count = MathTex(r"P(H)=\dfrac{"+str(Head)+r"}{"+str(Head+Tail)+r"}", font_size=34).next_to(grup, LEFT)
          for i in range(10):
            if Head == i+1:
              Number[i] += 1
            Height[i] = Number[i]/Total
            poly = [ax.coords_to_point(i,0,0),
                    ax.coords_to_point(i+1,0,0),
                    ax.coords_to_point(i+1,Height[i],0),
                    ax.coords_to_point(i,Height[i],0)]
            temp = Polygon(*poly).set_fill(BLUE, opacity=0.5)
            num = Tex(str(i+1), font_size=28).next_to(temp, UP, buff=0.1)
            rect[i] = VGroup(temp, num)
          NUM = Tex("N="+str(N+2), font_size=34).next_to(count, DOWN)
          self.add(rect, count, grup, NUM)
          self.wait(0.1)
        self.wait()


